In [59]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [60]:
import pandas as pd
import numpy as np
import pickle as pk
import re
import tensorflow as tf

from sentence_transformers import SentenceTransformer
from nltk import sent_tokenize
from sklearn.metrics.pairwise import cosine_similarity

import sys, os
sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname('__file__'))))

In [61]:
os.chdir('/content/drive/MyDrive/NLP/NLP-CaseLaw')

In [62]:
case_fraud = pd.read_csv('./Data/case_fraud.csv')
case_fraud.head(1)

,index,판례일련번호,사건명,사건번호,선고일자,법원명,사건종류명,사건종류코드,판결유형,선고,판례상세링크,판시사항,판결요지,참조조문,참조판례,판례내용
0,20,238021,특정경제범죄가중처벌등에관한법률위반(사기)·사기,2023도12424,2023.11.16,대법원,형사,400102.0,판결,선고,/DRF/lawService.do?OC=yeorii&target=prec&ID=23...,"유사수신행위를 금지·처벌하는 유사수신행위의 규제에 관한 법률 제6조 제1항, 제3...","유사수신행위의 규제에 관한 법률(이하 ‘유사수신행위법’이라 한다) 제6조 제1항, ...","유사수신행위의 규제에 관한 법률 제3조, 제6조 제1항, 형법 제37조, 제347조...",내용없음,【피 고 인】 피고인【상 고 인】 피고인【변 호 인】 변호사 장용배【배상신청인】 배...


# 0. 사건 Input

In [63]:
user_input = "저는 3대 독자인 남편과 혼인하여 3명의 딸을 두고 있어 아들 낳기를 고심하던 중 갑이 운영하는 병원에서 시술을 받으면\
    아들을 낳을 수 있다는 소문을 들었습니다. 저는 甲의 병원에 찾아가 갑에게 “아들을 낳고 싶다.”라고 하면서 甲으로부터 \
    일련의 시술과 처방을 받고 의료수가 및 약값 명목으로 금원을 지불하였으나 아무런 효과가 없었습니다. \
    이 경우 甲에게 형사상 죄책을 물을 수 있는지요?"

In [64]:
import re
from nltk import sent_tokenize


def prep(sent):
    sent = re.sub(r'\?', '.', sent)
    sent = re.sub('[^가-힣. ]', '', sent)

    return sent

def tokenize(sent):
    sent_tokened = sent_tokenize(sent)

    return sent_tokened

In [65]:
input_prep = prep(user_input)
input_tokened = tokenize(input_prep)
input_tokened

['저는 대 독자인 남편과 혼인하여 명의 딸을 두고 있어 아들 낳기를 고심하던 중 갑이 운영하는 병원에서 시술을 받으면    아들을 낳을 수 있다는 소문을 들었습니다.',
 '저는 의 병원에 찾아가 갑에게 아들을 낳고 싶다.라고 하면서 으로부터     일련의 시술과 처방을 받고 의료수가 및 약값 명목으로 금원을 지불하였으나 아무런 효과가 없었습니다.',
 '이 경우 에게 형사상 죄책을 물을 수 있는지요.']

# 2. SBERT

### 1.1 모델 & 임베딩 로드

In [66]:
with tf.device("/device:GPU:0"):
    sbert = SentenceTransformer('jhgan/ko-sbert-sts')

In [67]:
# 임베딩 값 로드
with open("./Modeling/model/sbert_emb.pickle", 'rb') as fr:
    sbert_embedding = pk.load(fr)

### 1.2 사용자 입력 임베딩

In [68]:
embedding = None
count = 0
for sentence in input_tokened:
    if embedding is None:
        embedding = sbert.encode(sentence)
    else:
        embedding += sbert.encode(sentence)
    count += 1

f_embedding = embedding / count

### 1.3 코사인 유사도 계산

In [69]:
from sklearn.metrics.pairwise import cosine_similarity

f_similarity = cosine_similarity([f_embedding], sbert_embedding)
f_similarity = list(enumerate(f_similarity[0]))
f_similarity = sorted(f_similarity, key=lambda x : x[1], reverse = True)

### 1.4 유사 판례 추출

In [70]:
precidents = []

for i in range(5):
    idx = f_similarity[i][0]
    sim = round(f_similarity[i][1] * 100, 2)

    precidents.append([case_fraud['판례일련번호'][idx], case_fraud['사건명'][idx], case_fraud['판례내용'][idx], sim])

In [71]:
f_result = pd.DataFrame(precidents, columns=['판례일련번호', '사건명', '판례내용', '유사도'])
f_result

,판례일련번호,사건명,판례내용,유사도
0,178184,사기·사문서위조·위조사문서행사,【피 고 인】 【상 고 인】 피고인들【변 호 인】 법무법인 서산 담당변호사 유진범【...,61.91
1,177835,사기,【피 고 인】 【상 고 인】 피고인【변 호 인】 변호사 이용명【원심판결】 부산지법 ...,61.69
2,237943,의료법위반·특정경제범죄가중처벌등에관한법률위반(사기)방조·의료법위반방조[적법하게 개설...,【피 고 인】 피고인 1 외 5인【상 고 인】 피고인들【변 호 인】 법무법인 우리하...,61.59
3,213717,사기ㆍ횡령[부동산 실권리자명의 등기에 관한 법률에 위반한 이른바 양자간 명의신탁에서...,[피고인] 피고인[상고인] 피고인 및 검사[변호인] 변호사 권종무[원심판결] 부산지...,61.58
4,184943,사기·배임,【피 고 인】 【상 고 인】 피고인 및 검사【변 호 인】 변호사 조기제【원심판결】 ...,61.12


In [73]:
f_result.판례내용[2]

'【피 고 인】 피고인 1 외 5인【상 고 인】 피고인들【변 호 인】 법무법인 우리하나로 외 1인【원심판결】 대구고법 2021. 12. 15. 선고 2021노115 판결【주    문】  상고를 모두 기각한다. 【이    유】  상고이유를 판단한다.   1. 피고인 1, 피고인 2, 피고인 3, 피고인 4, 피고인 5에 대한 의료법 위반 방조 부분, 피고인 의료법인 호암의료재단 부분  가. 관련 법리의료법인 명의로 개설된 의료기관을 실질적으로 비의료인이 개설·운영하였다고 판단하려면, 비의료인이 의료법인 명의 의료기관의 개설·운영에 주도적으로 관여하였다는 점을 기본으로 하여, 비의료인이 외형상 형태만을 갖추고 있는 의료법인을 탈법적인 수단으로 악용하여 적법한 의료기관 개설·운영으로 가장하였다는 사정이 인정되어야 한다. 이러한 사정은 비의료인이 실질적으로 재산출연이 이루어지지 않아 실체가 인정되지 아니하는 의료법인을 의료기관 개설·운영을 위한 수단으로 악용한 경우, 의료법인의 재산을 부당하게 유출하여 의료법인의 공공성, 비영리성을 일탈한 경우에 해당되면 인정될 수 있다(대법원 2023. 7. 17. 선고 2017도1807 전원합의체 판결 참조).   나. 판단  1) 원심은 다음과 같은 이유 등으로 피고인 의료법인 호암의료재단의 이사장인 원심 공동피고인 1이 이 사건 의료기관을 실질적으로 개설·운영하고 피고인 1, 피고인 2, 피고인 3, 피고인 4, 피고인 5가 이를 용이하게 하였다고 보아 이 부분 공소사실을 유죄로 판단한 제1심판결을 그대로 유지하였다.   가) 원심 공동피고인 1은 이 사건 의료기관의 개설·운영에 필요한 자금 대부분을 조달하고, 피고인 의료법인 호암의료재단(이하 ‘이 사건 의료법인’이라 한다)의 이사장에 취임하여 재정, 인사, 업무집행 등 이 사건 의료기관의 개설·운영에 관한 주요 사항을 주도적 입장에서 처리하였다.   나) 원심 공동피고인 1은 이 사건 의료법인 설립 당시 기본재산으로 가액 합계 약 35억 원의 부동산을, 보통재산으로 현금 5